In [3]:
import pandas as pd
import numpy as np
import datetime as dt
#from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.linear_model import CoxPHSurvivalAnalysis
from lifelines import CoxPHFitter

from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.exceptions import FitFailedWarning
import sksurv.util
import warnings
from lifelines.utils import k_fold_cross_validation

from sklearn.linear_model import LassoCV

from sksurv.preprocessing import OneHotEncoder


df=pd.read_csv("../1_data/private/preprocessed_data_Vitalant.csv")


FileNotFoundError: [Errno 2] No such file or directory: '../1_data/private/preprocessed_data_Vitalant.csv'

In [ ]:
#categorize some variables
pd.set_option('display.max_columns', 48)
df['DONOR_EDUCATION_DESC'].value_counts()
#Categorize donor birth country
df['DONOR_BORN_IN_US'] = np.where(df['DONOR_COUNTRY_OF_BIRTH']=='UNITED STATES (INCLUDING HAWAII)', 1, 0)

#Born in north america / mexico / india / china - look into counts

#categorise donor education into - post secondary, secondary (high school), or less than secondary or refused 

df.loc[(df['DONOR_EDUCATION_DESC']=='SOME COLLEGE OR TECHNICAL SCHOOL' )| (df['DONOR_EDUCATION_DESC']=="BACHELOR'S DEGREE")| 
      (df['DONOR_EDUCATION_DESC']=="MASTER'S OR PROFESSIONAL DEGREE")| (df['DONOR_EDUCATION_DESC']=="ASSOCIATE'S DEGREE")|  
       (df['DONOR_EDUCATION_DESC']=="GRADUATE DEGREE, MASTER'S")|(df['DONOR_EDUCATION_DESC']=="GRADUATE DEGREE, DOCTORATE")|
      (df['DONOR_EDUCATION_DESC']=="GRADUATE DEGREE, PROFESSIONAL")| (df['DONOR_EDUCATION_DESC']=="GRADUATE DEGREE, OTHER"),
       'DONOR_EDU']='Post Secondary'

df.loc[(df['DONOR_EDUCATION_DESC']=='HIGH SCHOOL GRAD-DIPLOMA OR EQUIV')| (df['DONOR_EDUCATION_DESC']==
        'HIGH SCHOOL GRADUATE OR EQUIVALENT'),'DONOR_EDU']='Secondary (High School)'

df.loc[(df['DONOR_EDUCATION_DESC']=='9th,10th,11th,12th GRADE-NO DIPLOMA' )| (df['DONOR_EDUCATION_DESC']=='8th GRADE OR LESS' )|
      (df['DONOR_EDUCATION_DESC']=='9TH, 10TH, 11TH, OR 12TH (NO DIPLOMA)' )| (df['DONOR_EDUCATION_DESC']=='8TH GRADE OR LESS'), 'DONOR_EDU']='Less than secondary'

df.loc[(df['DONOR_EDUCATION_DESC']=='REFUSED') | (df['DONOR_EDUCATION_DESC']=='UNAVAILABLE' )| (df['DONOR_EDUCATION_DESC']=='UNKNOWN') |  (df['DONOR_EDUCATION_DESC']=='NOT AVAILABLE'), 'DONOR_EDU']='UNAVAILABLE'

# make age categorical - high school age
df['high_school_age']=np.where(df['DONOR_AGE_AT_DONATION']<=20, 1, 0)

df.head()

In [2]:
#for cox fitter
df_cox2= df.loc[:,['time_to_return','CENSORED', 'DONATION_VOLUME_DRAWN', 'DONOR_GENDER', 'RACE_ETHNICITY','DONOR_BORN_IN_US', 'DONOR_EDU',
       'DONOR_WEIGHT', 'DONOR_HEIGHT', 'DONOR_AGE_AT_DONATION', 'Fixed_mobile', 'fixed_mobile_pattern',  'OUTCOME_TYPE', 'HGB_value','high_school_age','first_time','cum_lifetime_donations', 'unit_rbc_loss', 'rbc_loss_last_12_months','rbc_loss_last_24_months', 'days_since_last_rbc_loss', 'days_since_last_drbc_loss']]


X3=pd.get_dummies(df_cox2)
X3.fillna(0, inplace=True)
# add interactions
X3['first_time*fixed']=X3["first_time"]*X3['Fixed_mobile']
X3['first_time*hgb']=X3["first_time"]*X3['OUTCOME_TYPE_low hgb']
X3['hgb*fixed']=X3["OUTCOME_TYPE_low hgb"]*X3['Fixed_mobile']

y3= X3['CENSORED']

pd.set_option('display.max_columns', 48)
X3.head()


NameError: name 'df' is not defined

In [7]:

# Split the data into training and test sets
X_train3, X_test3, y_train3, y_test3 = train_test_split(X3, y3, test_size=0.5, random_state=0)

# Set up LassoCV model to find optimal penalty
model = LassoCV(cv=5, random_state=0)
model.fit(X_train3, y_train3)

# Get optimal penalty value
penalty = model.alpha_

print(penalty)

# Set up CoxPHFitter model with Lasso penalty
cph = CoxPHFitter(penalizer=penalty)

# Use k-fold cross-validation to fit the model
kf = KFold(n_splits=5, shuffle=True, random_state=0)
scores = []
for train_idx, val_idx in kf.split(X_train3):
    X_train_fold, X_val_fold = X_train3.iloc[train_idx], X_train3.iloc[val_idx]
    y_train_fold, y_val_fold = y_train3.iloc[train_idx], y_train3.iloc[val_idx]
    
    cph.fit(X_train_fold, duration_col='time_to_return', event_col='CENSORED')
    CV_score = cph.score(X_val_fold, scoring_method='concordance_index')
    scores.append(CV_score)

# Print average cross-validation score
print('Average cross-validation score:', sum(scores)/len(scores))


# Fit the model on the entire training set
cph.fit(X_train3, duration_col='time_to_return', event_col='CENSORED')

# Print summary of the model
cph.print_summary()

0.1814583716289931


/home/huzbah/.local/lib/python3.10/site-packages/lifelines/utils/__init__.py:1102: ConvergenceWarning: Column(s) ['cum_lifetime_donations', 'unit_rbc_loss', 'rbc_loss_last_12_months', 'rbc_loss_last_24_months', 'days_since_last_rbc_loss', 'days_since_last_drbc_loss', 'DONATION_PROCEDURE_TYPE_ND', 'DONOR_GENDER_UNKNOWN'] have very low variance. This may harm convergence. 1) Are you using formula's? Did you mean to add '-1' to the end. 2) Try dropping this redundant column before fitting if convergence fails.

  warnings.warn(dedent(warning_text), ConvergenceWarning)
/home/huzbah/.local/lib/python3.10/site-packages/lifelines/utils/__init__.py:800: RuntimeWarning: invalid value encountered in divide
  return (X - mean) / std
/home/huzbah/.local/lib/python3.10/site-packages/lifelines/utils/__init__.py:1122: ConvergenceWarning: Column DONOR_GENDER_UNKNOWN have very low variance when conditioned on death event present or not. This may harm convergence. This could be a form of 'complete separ

ConvergenceError: delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model

In [ ]:
from forestplot import forestplot
# Extract the coefficients and their confidence intervals

ci = cph.confidence_intervals_#.iloc[[2, 21, 22, 5, 4, 6, 23, 10, 16, 17, 14, 13, 11, 25, 19, 9, 24, 18, 15, 20, 1],:]

coef = cph.hazard_ratios_#[['Fixed_mobile', 'OUTCOME_TYPE_no draw', "OUTCOME_TYPE_other deferral",
                          # 'rbc_loss_last_12_months','unit_rbc_loss','rbc_loss_last_24_months', 
                           #'first_time*fixed',  'sex_M','race_Mixed Race',  'race_White', 
                           #'race_African Black', 'donation_product_x_WB', 'donation_product_x_DEF PROD',
                           #'hgb*fixed', 'OUTCOME_TYPE_completed', 'sex_F', 'first_time*hgb',
                           #'race_unknown','race_Asian', 'OUTCOME_TYPE_low hgb', 'first_time']]

covariate=coef.index

# Create a dataframe to store the coefficients and their confidence intervals
data = pd.DataFrame({'covariate': covariate, 'coef': coef, 'lower': ci['95% lower-bound'], 'upper': ci['95% upper-bound']})

data['lower'] = np.exp(data['lower'])
data['upper'] = np.exp(data['upper'])

# Sort the dataframe in descending order of coefficients
data = data.sort_values('coef', ascending=False)

# Create a numpy array of indices for each covariate
ind = np.arange(data.shape[0])


fp=forestplot(data,  # the dataframe with results data
              estimate="coef",  # col containing estimated effect size 
              ll="lower", hl="upper",  # columns containing conf. int. lower and higher limits
              varlabel="covariate",  # column containing variable label
              ylabel="Confidence interval",  # y-label title
              xlabel="Hazard Ratio",  # x-label title
             )
fp.axvline(x=1.00, linestyle='--', color='gray')
plt.show()